In [1]:
from src.returns_analysis import analyse_returns_characteristics
from src.process_selection import process_selection, recommend_process
from src.var import produce_var_results
from src.processes import Processes
from src.get_data import HistoricalData
from src.util import parse_json
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# config = parse_json("config.json")

# histdata = HistoricalData(config["binance"]["API_KEY"], config["binance"]["API_SECRET"])
# df = histdata.getKline("SOLUSDC","12h").dropna()
# histdata.save_data("test_data.csv")

In [3]:
df = pd.read_csv("test_data.csv").dropna()
df["opentime"] = pd.to_datetime(df.opentime)
df["closetime"] = pd.to_datetime(df.closetime)
df.head()

,opentime,open,high,low,close,volume,closetime,volume.1,log_returns
1,2025-06-27 00:08:00,139.30,139.38,139.28,139.31,216.126,2025-06-27 00:08:59.999,30115.31857,-0.000072
2,2025-06-27 00:09:00,139.33,139.35,139.25,139.30,11.725,2025-06-27 00:09:59.999,1633.54249,-0.000072
3,2025-06-27 00:10:00,139.29,139.30,139.23,139.30,55.019,2025-06-27 00:10:59.999,7662.20677,0.000000
4,2025-06-27 00:11:00,139.32,139.37,139.13,139.18,138.782,2025-06-27 00:11:59.999,19317.96890,-0.000862
5,2025-06-27 00:12:00,139.19,139.22,139.17,139.19,65.568,2025-06-27 00:12:59.999,9126.37733,0.000072


In [4]:
stats_summary = analyse_returns_characteristics(df)

BASIC STATS
mean : 3.196008531082171e-05
std : 0.0008471207457554803
Sharpe Ratio : 0.037727898261208356
skewness : 0.2616138307070276
kurtosis : 3.3191552334827312
jb_pvalue : 8.859698042676258e-103
min : -0.0044274291374359
max : 0.0054039205913282


In [5]:
test_results = process_selection(df)

In [6]:
recommend_process(test_results,stats_summary)

[{'process': 'GARCH',
  'confidence': 'high',
  'reason': 'significant volatility clustering detected'},
 {'process': 'jump diffusion (merton)',
  'confidence': 'medium',
  'reason': 'significant jump frequency'},
 {'process': 'Ornstein-Uhlenbeck',
  'confidence': 'medium',
  'reason': 'evidence of mean reversion in returns'},
 {'process': 'heston stochastic volatility',
  'confidence': 'medium',
  'reason': 'non-normal distrib with fat tails and skewness'}]

In [7]:
simulate_to = df.closetime.iloc[-1] + dt.timedelta(days = 5)
given_interval = ( df.closetime.iloc[1] - df.closetime.iloc[0] ).total_seconds()
periods = int((simulate_to - df.closetime.iloc[-1]).total_seconds() / given_interval)
processes = Processes(df,1234)

In [8]:
print(given_interval)
print((simulate_to - df.closetime.iloc[-1]).total_seconds())
print(periods)

60.0
432000.0
7200


GBM

In [9]:
price_path = processes.gbm_price_path(periods,1_000)
produce_var_results(price_path,df)

In [10]:
price_path

,sim_1,sim_2,sim_3,sim_4,sim_5,sim_6,sim_7,sim_8,sim_9,sim_10,...,sim_991,sim_992,sim_993,sim_994,sim_995,sim_996,sim_997,sim_998,sim_999,sim_1000
2025-06-27 16:46:59.999,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,...,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000,143.840000
2025-06-27 16:47:59.999,143.840095,143.839759,143.840290,143.839937,143.839854,143.840179,143.840174,143.839871,143.840003,143.839546,...,143.840093,143.840133,143.840152,143.839621,143.839887,143.840055,143.840188,143.840066,143.840150,143.839838
2025-06-27 16:48:59.999,143.839775,143.839431,143.840299,143.839597,143.840137,143.840009,143.840339,143.839861,143.840111,143.839383,...,143.840039,143.840380,143.840186,143.839370,143.839958,143.840183,143.840596,143.840224,143.840584,143.839654
2025-06-27 16:49:59.999,143.839535,143.839759,143.840291,143.839671,143.840503,143.840215,143.840660,143.840112,143.839914,143.839204,...,143.839891,143.840371,143.839968,143.839191,143.840086,143.840325,143.840375,143.840262,143.841126,143.839872
2025-06-27 16:50:59.999,143.839503,143.839747,143.840103,143.839856,143.840612,143.839847,143.840903,143.840303,143.839738,143.839501,...,143.839793,143.840525,143.839906,143.839148,143.839934,143.840383,143.840350,143.840404,143.841062,143.839919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 16:42:59.999,143.821565,143.848408,143.881738,143.856059,143.832765,143.860390,143.834512,143.874942,143.820674,143.843466,...,143.850966,143.872616,143.813490,143.822539,143.846407,143.827260,143.824070,143.830100,143.843331,143.844672
2025-07-02 16:43:59.999,143.821424,143.848616,143.881809,143.856148,143.832966,143.860263,143.834546,143.875005,143.821018,143.843601,...,143.850872,143.872147,143.813319,143.822357,143.846583,143.827737,143.823840,143.830030,143.843160,143.844964
2025-07-02 16:44:59.999,143.821327,143.848562,143.881858,143.856242,143.832941,143.860007,143.834773,143.875144,143.821263,143.843486,...,143.850730,143.872270,143.813185,143.822336,143.846446,143.827674,143.823843,143.829837,143.843345,143.844869
2025-07-02 16:45:59.999,143.821178,143.848740,143.881808,143.855995,143.833169,143.859973,143.834452,143.875469,143.820993,143.843629,...,143.850707,143.872320,143.813406,143.821969,143.846430,143.827394,143.823858,143.829793,143.843531,143.844717


Jump Diffusion Process

In [11]:
price_path = processes.jdp(24*60,1_000)
produce_var_results(price_path,df)

OU Process

In [12]:
price_path = processes.ou(24*60,1_000)
produce_var_results(price_path,df)

Fitted OU parameters: mu=3.20624132100886e-05, sigma=0.5104172777086196, theta=362.88
